In [1]:
import tensorflow as tf
import import_ipynb
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from keras import layers

from keras.layers import Input, Add, Dense, Dropout, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from keras.models import Model, load_model, Sequential

from tensorflow.keras.preprocessing import image
from tensorflow.keras.optimizers import RMSprop, Adam, SGD
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
import pydot
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
from resnets_utils import *
from keras.initializers import glorot_uniform
import scipy.misc
import cv2
import os 
from os import listdir
from os.path import isfile, join
import matplotlib.pyplot as plt
import imshow
%matplotlib inline

from keras import backend as K
K.set_image_data_format('channels_last')
K.set_learning_phase(1)

In [2]:
vgg16_model = tf.keras.applications.vgg16.VGG16()

553467904/553467096 [==============================] - 377s 1us/step


In [3]:
vgg16_model.summary()


Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [4]:
type(vgg16_model)


tensorflow.python.keras.engine.training.Model

In [5]:
model = Sequential()
for layer in vgg16_model.layers[:-1]:
    model.add(layer)

In [6]:
for layer in model.layers:
    layer.trainable = False

In [7]:
model.add(Dense(units=2, activation='softmax'))

In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)       2

In [9]:
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])


In [10]:
train = ImageDataGenerator(rescale= 1./255)

validation = ImageDataGenerator(rescale= 1./255)

test= ImageDataGenerator(rescale= 1./255)

In [11]:
train_dataset= train.flow_from_directory('basedata/training', 
                                         target_size= (224, 224),
                                         class_mode= 'categorical')

validation_dataset= validation.flow_from_directory('basedata/validation', 
                                                    target_size= (224, 224),
                                                    class_mode= 'categorical')  
test_dataset= test.flow_from_directory('basedata/test',
                                       target_size= (224, 224),
                                       class_mode= 'categorical') 


Found 1400 images belonging to 2 classes.
Found 200 images belonging to 2 classes.
Found 30 images belonging to 2 classes.


In [13]:
history= model.fit(train_dataset, 
                   steps_per_epoch= 25, 
                   validation_data= validation_dataset, 
                   validation_steps= 3,
                   shuffle=2, 
                   verbose=1, 
                   epochs = 20, 
                   batch_size = 32)

Epoch 1/20
25/25 [==============================] - 275s 11s/step - loss: 0.6840 - accuracy: 0.5391 - val_loss: 0.6697 - val_accuracy: 0.4792
Epoch 2/20
25/25 [==============================] - 270s 11s/step - loss: 0.6331 - accuracy: 0.6488 - val_loss: 0.6100 - val_accuracy: 0.6562
Epoch 3/20
25/25 [==============================] - 262s 10s/step - loss: 0.6053 - accuracy: 0.6780 - val_loss: 0.5859 - val_accuracy: 0.7188
Epoch 4/20
25/25 [==============================] - 263s 11s/step - loss: 0.5775 - accuracy: 0.6932 - val_loss: 0.5939 - val_accuracy: 0.6667
Epoch 5/20
25/25 [==============================] - 267s 11s/step - loss: 0.5588 - accuracy: 0.7050 - val_loss: 0.5830 - val_accuracy: 0.6771
Epoch 6/20
25/25 [==============================] - 262s 10s/step - loss: 0.5449 - accuracy: 0.7399 - val_loss: 0.5824 - val_accuracy: 0.6458
Epoch 7/20
25/25 [==============================] - 263s 11s/step - loss: 0.5586 - accuracy: 0.7000 - val_loss: 0.5629 - val_accuracy: 0.6875
Epoch 

KeyboardInterrupt: 